# FLSim Tutorial: Sentiment Classification with LEAF's Sent140



## Introduction

In this tutorial, we will train a binary sentiment classifier on LEAF's Sent140 dataset with federated learning using FLSim. 


### Prerequisites

To get the most of this tutorial, you should be comfortable with training machine learning models with **PyTorch** and familiar with the concept of **federated learning (FL)**. If you are unfamiliar with either of them or could use a refresher, please take a look at the following resources before proceeding with the tutorial:

- McMahan & Ramage (2017): [Federated Learning: Collaborative Machine Learning without Centralized Training Data](https://ai.googleblog.com/2017/04/federated-learning-collaborative.html). A short blog post from Google AI introducing the main idea of FL in a beginner-friendly way.
- McMahan et al. (2017): [Communication-Efficient Learning of Deep Networks from Decentralized Data](https://arxiv.org/pdf/1602.05629.pdf). This paper first proposes the approach of federated learning. The described algorithm is now known as federated averaging (or FedAvg for short).
- PyTorch has [extensive tutorials](https://pytorch.org/tutorials/) on their website.
- If you're new to **sentiment classification**, you can find Pang and Lee's survey on the topic [here](https://www.cs.cornell.edu/home/llee/omsa/omsa-published.pdf). 

Now that you're familiar with PyTorch and FL and have a sense of sentiment classification, let's move on!

### Objectives 

By the end of this tutorial, we will have learnt how to

1. Build a data pipeline for federated learning with FLSim,
2. Create a sentiment classification model compatible with FL training,
3. Create a metrics reporter to collect and report metrics,
4. Set hyperparameters for FL training, and
5. Launch an FL training flow using FLSim.

## Training a sentiment classifier with FLSim

### Prerequisites
First, let us install flsim via pip with the command below:

In [1]:
!pip install --quiet flsim

Some useful parameters for later - no need to change these.

In [56]:
USE_CUDA = True
LOCAL_BATCH_SIZE = 32
MAX_SEQ_LEN = 60

# suppress large outputs
VERBOSE = False

### 0. About the dataset

For this tutorial, we will use [LEAF's](https://leaf.cmu.edu/) [Sentiment140 (Sent140) dataset](https://leaf.cmu.edu/build/html/tutorials/sent140-md.html), which consists of 1.6 million tweets by 660k users. Note that the mean number of tweets per user is 2.42 and the standard deviation is 4.71.

![Sent140 distribution of samples across users](https://leaf.cmu.edu/webpage/images/twitter_hist.png)

Before the next step in this tutorial, we need to download the dataset and partition the data by users. 
In particular, we sample 
1% of the entire dataset (`--sf 0.01`) 
in a non-IID manner (`-s niid`) 
and partition 90% of sampled users into train and 10% of sampled users 
into test (`--tf 0.90`) as opposed to splitting individual samples into train and test (`-t 'user'`) .
We require all users to have at least one sample (`-k 1`).

For more information on the various preprocessing options, see [here](https://github.com/TalwalkarLab/leaf/tree/master/data/sent140). You can find the LEAF paper [here](https://arxiv.org/pdf/1812.01097.pdf).


In [3]:
%%capture preprocess_output

# Download and preprocess the data
!git clone https://github.com/TalwalkarLab/leaf.git
%cd leaf/data/shakespeare
!./preprocess.sh -s niid --sf 0.2 -k 0 -t sample -tf 0.8 --raw

In [4]:
if VERBOSE: print(preprocess_output)

Let us now find our preprocessed data. Note that if you use different preprocessing options, you might need to change these!



In [5]:
ALL_DATA = !ls data/all_data
ALL_DATA = "data/all_data/" + ALL_DATA[0]

In [6]:
TRAIN_DATA = !ls data/train
print(TRAIN_DATA)
TRAIN_DATA = "data/train/" + TRAIN_DATA[0]

TEST_DATA = !ls data/test
print(TRAIN_DATA)
TEST_DATA = "data/test/" + TEST_DATA[0]

TRAIN_DATA, TEST_DATA

['all_data_niid_2_keep_0_train_9.json']
data/train/all_data_niid_2_keep_0_train_9.json


('data/train/all_data_niid_2_keep_0_train_9.json',
 'data/test/all_data_niid_2_keep_0_test_9.json')

We can now load the training data and get a sense of how samples are distributed across users in our subset of the Sent140 dataset.


In [7]:
import json
import numpy as np

# load the training data
with open(ALL_DATA, "r") as f:
    training_data = json.load(f)

# how samples are distributed across users
n_samples = training_data['num_samples']
print(f"""\nNumber of samples per user:
  min={np.min(n_samples)}, 
  max={np.max(n_samples)}, 
  median={np.median(n_samples)}, 
  mean={np.mean(n_samples):.2f}, 
  std={np.std(n_samples):.2f}
  """)


Number of samples per user:
  min=3, 
  max=66903, 
  median=1163.0, 
  mean=3743.28, 
  std=6212.26
  


Let us also look at the data for an example user. Notice that there are multiple metadata fields in addition to the tweet itself and its sentiment label.

In [8]:
EXAMPLE_USER = training_data["users"][56]
print(training_data["user_data"][EXAMPLE_USER]["raw"])

Go, ye giddy goose.
                                            The music plays.
Then should you be nothing but musical; for you are
altogether govern'd by humours. Lie still, ye thief, and hear the
lady sing in Welsh.
Wouldst thou have thy head broken?
Then be still.
Now God help thee!
What's that?
Not mine, in good sooth.
I will not sing.



### 1. Data pipeline

Now, let us define how to build the data pipeline for federated learning:

1. To load the training and test data, we define a new dataset class, `ShakespeareDataset`, which converts each user verses into a `torch.Tensor`.



In [80]:
import itertools
import re
import string
import unicodedata

import torch
from torch.utils.data import Dataset


class ShakespeareDataset(Dataset):
    def __init__(self, data_root, max_seq_len):
        self.data_root = data_root
        self.max_seq_len = max_seq_len
        self.char2idx = {c: i for i, c in enumerate(string.printable)}
        self.idx2char = {i: u for i, u in enumerate(string.printable)}
        self.num_letters = len(self.char2idx)
        self.UNK = self.num_letters

        with open(data_root, "r+") as f:
            self.dataset = json.load(f)

        self.data = {}
        self.targets = {}

        # Populate self.data and self.targets
        for user_id, user_data in self.dataset["user_data"].items():
            self.data[user_id] = self.process_x(list(user_data["x"]))
            self.targets[user_id] = self.process_y(list(user_data["y"]))

    def __len__(self):
        return len(self.data)

    def __iter__(self):
        for user_id in self.data.keys():
            yield self.__getitem__(user_id)

    def __getitem__(self, user_id: str):
        if user_id not in self.data or user_id not in self.targets:
            raise IndexError(f"User {user_id} is not in dataset")
        return self.data[user_id], self.targets[user_id]

    def line_to_indices(self, line: str, max_seq_len: int):
        line_list = self.split_line(line)  # split phrase in words
        line_list = line_list
        chars = self.flatten_list([list(word) for word in line_list])
        indices = [self.char2idx[letter]
                   if letter in self.char2idx else self.UNK 
                   for i, letter in enumerate(chars) if i < max_seq_len
          ]
        # Add padding
        indices = indices + [self.UNK] * (max_seq_len - len(indices))
        return indices

    def process_x(self, raw_x_batch):
        x_batch = [self.line_to_indices(e, self.max_seq_len) for e in raw_x_batch]
        x_batch = torch.LongTensor(x_batch)
        return x_batch

    def process_y(self, raw_y_batch):
        y_batch = [self.char2idx[e] for e in raw_y_batch]
        return y_batch

    def split_line(self, line):
        """
        Split given line/phrase (str) into list of words (List[str])
        """
        return re.findall(r"[\w']+|[.,!?;]", line)

    def flatten_list(self, nested_list):
        return list(itertools.chain.from_iterable(nested_list))


2. We can now load the train and test dataset using `ShakespeareDataset`.


In [81]:
# 2. Load the train and test datasets.
train_dataset = ShakespeareDataset(
    data_root=TRAIN_DATA,
    max_seq_len=MAX_SEQ_LEN,
)

In [82]:
test_dataset = ShakespeareDataset(
    data_root=TEST_DATA,
    max_seq_len=MAX_SEQ_LEN,
)

Recall our `EXAMPLE_USER` from earlier? Their data now looks like this:

In [83]:
for user in train_dataset:
  print(user)
  break

(tensor([[43, 14, 27,  ..., 12, 17, 24],
        [14, 27, 14,  ..., 17, 24, 18],
        [27, 14, 73,  ..., 24, 18, 12],
        ...,
        [18, 23, 29,  ..., 29, 27, 14],
        [18, 23, 29,  ..., 29, 27, 14],
        [23, 29, 17,  ..., 27, 14, 29]]), [32, 17, 18, 12, 17, 94, 34, 24, 30, 27, 94, 43, 18, 16, 17, 23, 14, 28, 28, 94, 32, 18, 21, 21, 94, 28, 14, 14, 94, 15, 18, 27, 28, 29, 75, 94, 84, 42, 18, 31, 18, 23, 16, 94, 10, 94, 25, 10, 25, 14, 27, 86, 94, 36, 94, 25, 21, 10, 34, 94, 29, 17, 14, 27, 14, 94, 18, 28, 73, 94, 22, 34, 94, 21, 24, 27, 13, 73, 94, 28, 24, 22, 14, 94, 29, 14, 23, 94, 32, 24, 27, 13, 28, 94, 21, 24, 23, 16, 73, 94, 58, 17, 18, 12, 17, 94, 18, 28, 94, 10, 28, 94, 11, 27, 18, 14, 15, 94, 10, 28, 94, 44, 94, 17, 10, 31, 14, 94, 20, 23, 24, 32, 23, 94, 10, 94, 25, 21, 10, 34, 78, 94, 37, 30, 29, 94, 11, 34, 94, 29, 14, 23, 94, 32, 24, 27, 13, 28, 73, 94, 22, 34, 94, 21, 24, 27, 13, 73, 94, 18, 29, 94, 18, 28, 94, 29, 24, 24, 94, 21, 24, 23, 16, 73, 94, 58,

To complete our data pipeline, we only need to

3. Create a data loader, which will batchify training, eval, and test data. There is no need to create a sharder since each data sample is already associated with a user. For each dataset, the data loader splits each client's data into batches of size `batch_size`. We choose not to drop the last batch.

4. Lastly, wrap the data loader with a data provider and return it. 
The data provider creates clients from the groupings in the data loader and adds metadata (e.g. number of examples, number of batches per client). 
Our data is now formatted such that the trainer will accept it.

In [84]:
from flsim.utils.example_utils import LEAFDataLoader, DataProvider

# 3. Batchify training, eval, and test data. Note that train_dataset is already sharded.
dataloader = LEAFDataLoader(
    train_dataset,
    test_dataset,
    test_dataset,
    batch_size=LOCAL_BATCH_SIZE,
    drop_last=False,
)

# 4. Wrap the data loader with a data provider.
data_provider = DataProvider(dataloader)
print(f"\nClients in total: {data_provider.num_train_users()}")

Creating FL User: 133user [00:04, 30.13user/s]
Creating FL User: 133user [00:00, 1079.14user/s]
Creating FL User: 133user [00:00, 446.02user/s] 


Clients in total: 133


### 2. Create the model

Now, let's see how we can create a model that is compatible with FL-training.

1. First, we define a standard, non-FL sentiment classification PyTorch `nn.Module`. In this tutorial we use a simple char-LSTM with an embedding, LSTM, and linear layer.

2. Create a `torch.device` and choose where the model will be allocated (CUDA or CPU).

As with the data pipeline, these steps are identical to creating a model in non-federated learning. Note that in contrast to non-FL learning, we haven't moved the model to device yet.

In [12]:
from torch import nn

class CharLSTM(nn.Module):
    def __init__(
        self,
        num_classes,
        n_hidden,
        num_embeddings,
        embedding_dim,
        max_seq_len,
        dropout_rate,
    ):
        super().__init__()
        self.dropout_rate = dropout_rate
        self.n_hidden = n_hidden
        self.num_classes = num_classes
        self.max_seq_len = max_seq_len
        self.num_embeddings = num_embeddings

        self.embedding = nn.Embedding(
            num_embeddings=self.num_embeddings, embedding_dim=embedding_dim
        )
        self.lstm = nn.LSTM(
            input_size=embedding_dim,
            hidden_size=self.n_hidden,
            num_layers=2,
            batch_first=True,
            dropout=self.dropout_rate,
        )
        self.fc = nn.Linear(self.n_hidden, self.num_classes)
        self.dropout = nn.Dropout(p=self.dropout_rate)

    def forward(self, x):
        seq_lens = torch.sum(x != (self.num_embeddings - 1), 1) - 1
        x = self.embedding(x)  # [B, S] -> [B, S, E]
        out, _ = self.lstm(x)  # [B, S, E] -> [B, S, H]
        out = out[torch.arange(out.size(0)), seq_lens]
        out = self.fc(self.dropout(out))  # [B, S, H] -> # [B, S, C]
        return out

We initialize our model with such parameters that it is compatible with our dataset.

In [13]:
import torch

# 1. Define our model, a simple char-LSTM.
model = CharLSTM(
    num_classes=train_dataset.num_classes,
    n_hidden=100,
    num_embeddings=train_dataset.num_letters + 1,
    embedding_dim=100,
    max_seq_len=MAX_SEQ_LEN,
    dropout_rate=0.1,
)

# 2. Choose where the model will be allocated.
cuda_enabled = torch.cuda.is_available() and USE_CUDA
device = torch.device(f"cuda:{0}" if cuda_enabled else "cpu")

model, device

(CharLSTM(
   (embedding): Embedding(101, 100)
   (lstm): LSTM(100, 100, num_layers=2, batch_first=True, dropout=0.1)
   (fc): Linear(in_features=100, out_features=2, bias=True)
   (dropout): Dropout(p=0.1, inplace=False)
 ), device(type='cpu'))

As with the data pipeline, there are a few extra steps that we need to take to make sure that our model is compatible with FL. In particular, we need to

3. Wrap the PyTorch module with the FLSim `FLModel`, an abstracted version of a FL-friendly model class that is accepted by the trainer and handles metric collection, as well as the forward pass for both training and evaluation. We can recover our `nn.Module` by calling `FLModel.fl_get_module()`

4. Move the model to GPU and enable CUDA if desired. `FLModel.fl_cuda()` internally calls `model.to(device)` to move the model to GPU.

In [14]:
from flsim.utils.example_utils import FLModel

# 3. Wrap the model with FLModel.
global_model = FLModel(model, device)
assert(global_model.fl_get_module() == model)

# 4. Move the model to GPU and enable CUDA if desired.
if cuda_enabled:
    global_model.fl_cuda()

### 3. Metrics Reporting

After having created our data pipeline and FL model, we will now create our metrics reporter. 
The metrics reporter allows us to collect, evaluate, and report relevant training, aggregation, and evaluation/test metrics as well as log them onto TensorBoard.



In [15]:
from flsim.interfaces.metrics_reporter import Channel
from flsim.utils.example_utils import MetricsReporter

# Create a metric reporter.
metrics_reporter = MetricsReporter([Channel.TENSORBOARD, Channel.STDOUT])

There are three functions that are of particular interest:

1. `compute_scores` computes the metrics of interest for both training and aggregation (if desired) as well as evaluation/test.

2. `create_eval_metrics` creates a dictionary that stores the value for each eval metric. 

3. `compare_metrics` compares the current eval metrics that are returned by `create_eval_metrics` to the best eval metrics so far.

For this tutorial, our only metric of interest is top-1 accuracy. In general, as with the data loading and model, you should write your own metrics reporter depending on the task.

In [16]:
import inspect

if VERBOSE:
    print(inspect.getsource(MetricsReporter.compute_scores))
    print(inspect.getsource(MetricsReporter.create_eval_metrics))
    print(inspect.getsource(MetricsReporter.compare_metrics))

### 4. Hyperparameters

We can represent the hyperparameters for FL training in a JSON config for ease of representation and we convert the JSON config to OmegaConf before passing it to the FL trainer.

In particular, we specify a FedAvg with LR implementation with 10 users per round.

In [17]:
import flsim.configs
from flsim.utils.config_utils import fl_config_from_json
from omegaconf import OmegaConf

json_config = {
    "trainer": {
        "_base_": "base_sync_trainer",
        "server": {
            "_base_": "base_sync_server",
            "server_optimizer": {
                # there are different types of server optimizers
                # fed avg with lr requires a learning rate, whereas e.g. fed_avg doesn't
                "_base_": "base_fed_avg_with_lr",
                # server's learning rate
                "lr": 0.7,
                # server's global momentum
                "momentum": 0.9
            },
            # aggregate client models into a single model by taking their weighted sum
            "aggregation_type": "WEIGHTED_AVERAGE",
            # type of user selection sampling
            "active_user_selector": {
                "_base_": "base_uniformly_random_active_user_selector"
            }
        },
        "client": {
            # number of client's local epochs
            "epochs": 1,
            "optimizer": {
                # client's optimizer
                "_base_": "base_optimizer_sgd",
                # client's local learning rate
                "lr": 1,
                # client's local momentum
                "momentum": 0
            }
        },
        # number of users per round for aggregation
        "users_per_round": 10,
        # total number of global epochs
        # total #rounds = ceil(total_users / users_per_round) * epochs
        "epochs": 1,
        # frequency of reporting train metrics
        "train_metrics_reported_per_epoch": 10,
        # keep the trained model always (as opposed to only when it
        # performs better than the previous model on eval)
        "always_keep_trained_model": False,
        # frequency of evaluation per epoch
        "eval_epoch_frequency": 1,
        "do_eval": True,
        # should we report train metrics after global aggregation
        "report_train_metrics_after_aggregation": True
    }
}

cfg = fl_config_from_json(json_config)
if VERBOSE: print(OmegaConf.to_yaml(cfg))

/usr/local/lib/python3.7/dist-packages/hydra/experimental/initialize.py:36: UserWarning: hydra.experimental.initialize() is no longer experimental. Use hydra.initialize()
  message="hydra.experimental.initialize() is no longer experimental."
/usr/local/lib/python3.7/dist-packages/hydra/experimental/compose.py:19: UserWarning: hydra.experimental.compose() is no longer experimental. Use hydra.compose()
  message="hydra.experimental.compose() is no longer experimental."


### 5. Training
Recall that we already built the data provider and created a model compatible with FL training. 
We also initialized a metrics reporter and set our desired hyperparameters.

Now, we only need to instantiate the trainer with the model and hyperparameter config we defined earlier to launch the FL training flow. We run FL training with the above JSON config and utilize `eval_score` to store the final evaluation metrics.



In [18]:
from hydra.utils import instantiate

# Instantiate the trainer.
trainer = instantiate(cfg.trainer, model=global_model, cuda_enabled=cuda_enabled)   

# Launch FL training.
final_model, eval_score = trainer.train(
    data_provider=data_provider,
    metrics_reporter=metrics_reporter,
    num_total_users=data_provider.num_train_users(),
    distributed_world_size=1,
)

Round:  10%|█         | 59/588 [00:26<03:53,  2.26round/s]

Train finished Global Round: 59
(epoch = 1, round = 59, global round = 59), Loss/Training: 0.8090709392856431
(epoch = 1, round = 59, global round = 59), Accuracy/Training: 51.68040583386176
reporting (epoch = 1, round = 59, global round = 59) for aggregation
(epoch = 1, round = 59, global round = 59), Loss/Aggregation: 0.684047845005989
(epoch = 1, round = 59, global round = 59), Accuracy/Aggregation: 61.111111111111114


Round:  20%|██        | 118/588 [00:51<03:30,  2.24round/s]

Train finished Global Round: 118
(epoch = 1, round = 118, global round = 118), Loss/Training: 0.7293585050800765
(epoch = 1, round = 118, global round = 118), Accuracy/Training: 51.34589502018842
reporting (epoch = 1, round = 118, global round = 118) for aggregation
(epoch = 1, round = 118, global round = 118), Loss/Aggregation: 0.7340972363948822
(epoch = 1, round = 118, global round = 118), Accuracy/Aggregation: 53.333333333333336


Round:  30%|███       | 177/588 [01:15<02:54,  2.36round/s]

...

Round:  90%|█████████ | 530/588 [03:41<00:25,  2.29round/s]

Train finished Global Round: 530
(epoch = 1, round = 530, global round = 530), Loss/Training: 0.7606214851493368
(epoch = 1, round = 530, global round = 530), Accuracy/Training: 51.7467248908297
reporting (epoch = 1, round = 530, global round = 530) for aggregation
(epoch = 1, round = 530, global round = 530), Loss/Aggregation: 0.5512452930212021
(epoch = 1, round = 530, global round = 530), Accuracy/Aggregation: 69.23076923076923


Round: 100%|█████████▉| 587/588 [04:06<00:00,  2.23round/s]

Train finished Global Round: 588
(epoch = 1, round = 588, global round = 588), Loss/Training: 0.7553911558512983
(epoch = 1, round = 588, global round = 588), Accuracy/Training: 46.5288035450517
reporting (epoch = 1, round = 588, global round = 588) for aggregation
(epoch = 1, round = 588, global round = 588), Loss/Aggregation: 0.6767090618610382
(epoch = 1, round = 588, global round = 588), Accuracy/Aggregation: 57.142857142857146
Running (epoch = 1, round = 588, global round = 588) for Eval
(epoch = 1, round = 588, global round = 588), Loss/Eval: 0.8684062315448778
(epoch = 1, round = 588, global round = 588), Accuracy/Eval: 49.61439588688946


Epoch:   0%|          | 0/1 [04:08<?, ?epoch/s]


After training finishes, we evaluate the model and report the accuracy on the test set before finishing this tutorial.


In [19]:
# We can now test our trained model.
trainer.test(
    data_provider=data_provider,
    metrics_reporter=MetricsReporter([Channel.STDOUT]),
)

Running (epoch = 1, round = 1, global round = 1) for Test
(epoch = 1, round = 1, global round = 1), Loss/Test: 0.8683878255035598
(epoch = 1, round = 1, global round = 1), Accuracy/Test: 49.61439588688946


{'Accuracy': 49.61439588688946}

## Summary

In this tutorial, we first showed how to get and preprocess LEAF's Sent140 dataset. 
We then built a data provider by splitting each client's data into batches. 
We defined a simple char-LSTM as our model, wrapped it with a model compatible with FL training, and moved it to GPU. 
Lastly, we set the hyperparameters for FL training, launched the training flow, and evaluated our model.

### Additional resources

- For a more in-depth understanding of this tutorial, check out [example_utils.py](https://github.com/facebookresearch/FLSim/blob/main/flsim/utils/example_utils.py) where we define the data loader, data provider, `FLModel`, and metrics reporter that we use in this tutorial.

- [FLSim tutorials](https://github.com/facebookresearch/FLSim/tree/main/tutorials) - check out our other tutorial on image classification.

- Kairouz et al. (2021): [Advances and Open Problems in Federated Learning](https://arxiv.org/pdf/1912.04977.pdf). As the title suggests, an in-depth overview of advances and open problems in FL.

- If you're interested in federated learning with differential privacy, take a look at [Opacus](https://opacus.ai/), a library that enables training PyTorch models with differential privacy. 
You can find a blog post introducing Opacus [here](https://ai.facebook.com/blog/introducing-opacus-a-high-speed-library-for-training-pytorch-models-with-differential-privacy/).

